# Fake news predictor using Logistic Regression
- Logistic Regression since this is a binary classification

In [2]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anushthan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
news_dataset = pd.read_csv('./Data/news_train.csv')
news_dataset.shape

(20800, 5)

In [5]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
# Replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [8]:
# Merging the author name, news title and news article
news_dataset['content'] = news_dataset['title']+' '+news_dataset['author']+' '+news_dataset['text']

news_dataset['content'].head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2    Why the Truth Might Get You Fired Consortiumne...
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: content, dtype: object

## Stemming our data

In [10]:
port_stem = PorterStemmer()

In [11]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [12]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [13]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [18]:
# Converting text to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [19]:
print(X)

  (0, 109752)	0.04915831242516885
  (0, 109697)	0.019064671151527695
  (0, 108742)	0.044165441199081334
  (0, 108738)	0.09477494042884231
  (0, 108695)	0.037584880979390035
  (0, 108658)	0.011306147740716936
  (0, 108007)	0.017092546683505853
  (0, 107190)	0.017105936674103112
  (0, 107099)	0.012543234221230961
  (0, 107013)	0.02912641710492832
  (0, 106934)	0.012863319680563094
  (0, 106734)	0.011771716334271504
  (0, 105884)	0.025727197929110483
  (0, 105848)	0.03129670137812476
  (0, 104837)	0.021536495542122617
  (0, 103422)	0.0654455539825981
  (0, 102736)	0.03314918847150755
  (0, 102485)	0.016396128180984536
  (0, 101717)	0.03807192497938021
  (0, 101077)	0.01108240343647574
  (0, 101067)	0.04320446706289209
  (0, 101014)	0.13602128375819164
  (0, 100866)	0.07130923370634748
  (0, 99577)	0.039449889166193734
  (0, 99009)	0.02712035892973115
  :	:
  (20799, 7470)	0.010635431711878486
  (20799, 7143)	0.02816704434978389
  (20799, 6848)	0.03959171777516513
  (20799, 6810)	0.0253655

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [21]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [22]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9772235576923077


In [23]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9612980769230769


In [ ]:
# Dumping the model in a pickle file for future use
import pickle
file_path = './Models/news_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model, file)

# Prediction on custom data

In [28]:
X_new = X_test[3]

prediction = model.predict(X_new)
print("Fake" if prediction[0]==0 else "Real")

Fake
